In [4]:
import numpy as np
import pandas as pd
import requests 
import xlsxwriter
import math

In [5]:
stocks = pd.read_csv("sp_500_stocks.csv")
from API_token import IEX_CLOUD_API_TOKEN
def chunks(lst, n):
    # Yield successive n-sized chungs from list
    for i in range(0, len(lst), n):
        yield lst[i:i+n]
        
# IEX Cloud limits batch API calls to 100 tickers per request to boost up the time 
# Splitting into groups of 100 tickers
symbol_groups = list(chunks(stocks["Ticker"], 100))

In [6]:
# Information from the API
# Market cap for each stock and price of each stock

symbol_strings = []
# creating strings of symbol for batch API calls
for i in range(len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))

columns = ["Ticker", "Price", "Market Cap", "Number of Shares to Buy"]
finalDF = pd.DataFrame(columns=columns)
# parsing the data to input into the final dataframe
for symbols in symbol_strings:
    batch_api_call = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call).json()
    for symbol in symbols.split(","):
        finalDF = finalDF.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]["quote"]["latestPrice"],
                    data[symbol]["quote"]["marketCap"],
                    "N/A"
                ],
            index = columns
            ),
            ignore_index = True
        )

In [13]:
while True:
    # Enter integer value from the console.
    portfolio_size = input('Enter the value of your portfolio: ')

    try: 
        val = float(portfolio_size)
    except ValueError:
        print("Please enter a number")
    else:
        break
        
portfolio_size = val/len(finalDF.index)
for i in range(len(finalDF.index)):
    finalDF.loc[i, "Number of Shares to Buy"] = portfolio_size // finalDF.loc[i, "Price"]
    

Enter the value of your portfolio: 10000000


In [16]:
finalDF

In [25]:
# Writing to excel
writer = pd.ExcelWriter("RecommendedTrades.xlsx", engine="xlsxwriter")
finalDF.to_excel(writer, "RecommendedTrades", index = False)

# setting background and font color
background_color = "#0a0a23"
font_color = "#ffffff"

# String format for ticker
# $XX.XX format for stock price
# $XX,XXX format for market cap
# integer format for the number of shares purchase
string_format = writer.book.add_format(
    {
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

dollar_format = writer.book.add_format(
    {
        "num_format": "$0.00",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

integer_format = writer.book.add_format(
    {
        "num_format": "0",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

column_format = {
    "A": ["Ticker", string_format],
    "B": ["Price", dollar_format],
    "C": ["Market Capitalization", dollar_format],
    "D": ["Number of Shares to Buy", integer_format]
}

# Formatting each column to its readable form 
# and reformat the column headers    
for column in column_format.keys():
    writer.sheets["RecommendedTrades"].set_column(f"{column}:{column}", 18, column_format[column][1]) # 18 is the pixel for the size of the column
    writer.sheets["RecommendedTrades"].write(f"{column}1", column_format[column][0], column_format[column][1])
    



writer.save()



